In [3]:
exec(open("init_notebook.py", "r").read())

current working dir/home/abjawad/Documents/GitHub/local-attention-model


In [4]:
from models.swin_transformer import PatchEmbed
from models.encoders.student_segformer import OverlapPatchEmbed, Mlp, DWConv, Block
from models.encoders.merge_attn import MultiScaleAttention
import torch

[Warning] Fused window process have not been installed. Please refer to get_started.md for installation.


In [5]:
swin_patch_embed = PatchEmbed(img_size=224, 
                              patch_size=4, 
                              in_chans=3, 
                              embed_dim=96, 
                              norm_layer=None)

overlap_patch_embed = OverlapPatchEmbed(patch_size=7, 
                                        stride=4, 
                                        in_chans=3, 
                                        embed_dim=64)

mlp = Mlp(in_features=64,
          hidden_features=64*4,
          act_layer=torch.nn.GELU,
          drop=0)

multiscale_attn = MultiScaleAttention(dim=64,
                                      num_heads=2,
                                      sr_ratio=8,
                                      local_region_shape=[8, 16],
                                      img_size=(256, 256)) 

block = Block(dim=64,
              num_heads=2, 
              mlp_ratio=4,
              qkv_bias=True,
              qk_scale=None,
              drop=0,
              attn_drop=0, 
              drop_path=0,
              norm_layer=torch.nn.LayerNorm,
              sr_ratio=8,
              local_region_shape=[8, 16],
              img_size=(256, 256))




merge attn multiscale attention  64 2 False None 0.0 0.0 8 [8, 16] (256, 256)
merge attn multiscale attention  64 2 True None 0 0 8 [8, 16] (256, 256)
====== block ====== 64 2 4 True None 0 0 0 <class 'torch.nn.modules.activation.GELU'> <class 'torch.nn.modules.normalization.LayerNorm'> 8 [8, 16] (256, 256)


In [6]:
# craete random input and pass through the model
x_1 = torch.rand(1, 3, 224, 224)
y_1 = swin_patch_embed(x_1)

x_2 = torch.rand(1, 3, 1024, 1024)
y_2 = overlap_patch_embed(x_2)

x_3, h, w = torch.rand(1, 65536, 64), 256, 256
y_3 = mlp(x_3, h, w)

x_4 = torch.rand(1, 65536, 64)
y_4 = multiscale_attn(x_4, 256, 256)

x_5 = torch.rand(1, 65536, 64)
y_5 = block(x_5, 256, 256)

print(swin_patch_embed.flops())
print(overlap_patch_embed.flops())
print(mlp.flops())
print(multiscale_attn.flops())


 %%%% x proj: torch.Size([1, 96, 56, 56])%%%%%%%
x flatten: torch.Size([1, 96, 3136])
input: MLP  torch.Size([1, 65536, 64]) 256 256
!!!!!!!!!!!!attention head:  2  !!!!!!!!!!
torch.Size([1, 65536, 64]) 256 256
!!!!!!!!!!!!attention head:  2  !!!!!!!!!!
torch.Size([1, 65536, 64]) 256 256
input: MLP  torch.Size([1, 65536, 64]) 256 256
14450688
616562688.0
4596957184
40640


##### self.patch_embed1 = OverlapPatchEmbed(patch_size=7, stride=4, in_chans=in_chans, embed_dim=64)

##### input:  torch.Size([1, 3, 1024, 1024])
##### ############### Stage 1 ##########################
##### tokenization:  torch.Size([1, 65536, 64])
##### output:  torch.Size([1, 64, 256, 256])
##### ******** End Stage 1 **************
##### ############### Stage 2 ##########################
##### :  torch.Size([1, 16384, 128])
##### output:  torch.Size([1, 128, 128, 128])
##### ******** End Stage 2 **************
##### ############### Stage 3 ##########################
##### tokenization:  torch.Size([1, 4096, 320])
##### output:  torch.Size([1, 320, 64, 64])
##### ******** End Stage 3 **************
##### ############### Stage 4 ##########################
##### tokenization:  torch.Size([1, 1024, 512])
##### output:  torch.Size([1, 512, 32, 32])
##### ******** End Stage 4 **************